<a href="https://colab.research.google.com/github/uol-mediaprocessing/group-projects-image-restauration/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Restoration of Damaged Images

Gruppenmitglieder:

- Sören Gade
- Florian Meyer

## Ziele

Folgende Zielsetzung soll bis zum Ende der Modullaufzeit erreicht werden:

1. _Ziel 1_
2. _Ziel 2_
3. _Ziel 3_
4. _Ziel 4_

## Setup

In [0]:
! wget https://raw.githubusercontent.com/uol-mediaprocessing/notebooks/master/pictures/Lenna.png

![Lenna](Lenna.png)